In [ ]:
import numpy as np
import re

# Original text data
data = """Deep learning (also known as deep structured learning) is part of a broader family of machine learning methods based on artificial neural networks with representation learning. Learning can be supervised, semi-supervised or unsupervised. Deep-learning architectures such as deep neural networks, deep belief networks, deep reinforcement learning, recurrent neural networks, convolutional neural networks and Transformers have been applied to fields including computer vision, speech recognition, natural language processing, machine translation, bioinformatics, drug design, medical image analysis, climate science, material inspection and board game programs, where they have produced results comparable to and in some cases surpassing human expert performance."""

# Splitting the data into sentences
sentences = data.split('.')
clean_sent = []

# Cleaning and preprocessing each sentence
for sentence in sentences:
    if sentence == "":
        continue
    # Remove non-alphanumeric characters
    sentence = re.sub('[^A-Za-z0-9]+', ' ', sentence)
    # Remove single characters and convert to lowercase
    sentence = re.sub(r'(?:^| )\w (?:$| )', ' ', sentence).strip()
    sentence = sentence.lower()
    clean_sent.append(sentence)

# Tokenizing sentences
from tensorflow.keras.preprocessing.text import Tokenizer
tokenizer = Tokenizer()
tokenizer.fit_on_texts(clean_sent)

# Converting text to sequences of integers
sequences = tokenizer.texts_to_sequences(clean_sent)
print(sequences)

# Creating word-index and index-word dictionaries
index_to_word = {}
word_to_index = {}
for i, sequence in enumerate(sequences):
    word_in_sentence = clean_sent[i].split()
    for j, value in enumerate(sequence):
        index_to_word[value] = word_in_sentence[j]
        word_to_index[word_in_sentence[j]] = value

print(index_to_word, "\n")
print(word_to_index)

# Setting vocabulary size and embedding dimensions
vocab_size = len(tokenizer.word_index) + 1
emb_size = 10
context_size = 2

# Generating context and target pairs
contexts = []
targets = []
for sequence in sequences:
    for i in range(context_size, len(sequence) - context_size):
        target = sequence[i]
        # Context words are the two words before and after the target word
        context = [sequence[i - 2], sequence[i - 1], sequence[i + 1], sequence[i + 2]]
        contexts.append(context)
        targets.append(target)

print(contexts, "\n")
print(targets)

# Displaying some sample features with targets
for i in range(5):
    words = []
    target = index_to_word.get(targets[i])
    for j in contexts[i]:
        words.append(index_to_word.get(j))
    print(words, " -> ", target)

# Convert the contexts and targets to numpy arrays
X = np.array(contexts)
Y = np.array(targets)

# Importing necessary libraries for building the model
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, Lambda

# Define the model
model = Sequential([
    # Embedding layer to learn word embeddings
    Embedding(input_dim=vocab_size, output_dim=emb_size, input_length=2*context_size),
    # Lambda layer to average embeddings for each context
    Lambda(lambda x: tf.reduce_mean(x, axis=1)),
    # Hidden layers
    Dense(256, activation='relu'),
    Dense(512, activation='relu'),
    # Output layer with softmax activation for multi-class classification
    Dense(vocab_size, activation='softmax')
])

# Compile the model
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
history = model.fit(X, Y, epochs=80)

# Plotting the model's training history (accuracy/loss)
import seaborn as sns
sns.lineplot(history.history['loss'], label='Training Loss')
sns.lineplot(history.history['accuracy'], label='Training Accuracy')

# Reducing word embedding dimensions for visualization
from sklearn.decomposition import PCA
embeddings = model.get_weights()[0]
pca = PCA(n_components=2)
reduced_embeddings = pca.fit_transform(embeddings)

# Test the model with example sentences
test_sentences = [
    "known as structured learning",
    "transformers have applied to",
    "where they produced results",
    "cases surpassing expert performance"
]

for sent in test_sentences:
    test_words = sent.split(" ")
    x_test = [word_to_index.get(i) for i in test_words]
    x_test = np.array([x_test])

    # Predict the next word based on context
    pred = model.predict(x_test)
    pred = np.argmax(pred[0])
    print("Prediction for", test_words, "->", index_to_word.get(pred), "\n")
